In [19]:
# =============================================================================
# Project: Visual Attribute Analyzer
# Description: Automated extraction of structured visual metadata and dimension
#              scores from product images using Multimodal AI (Google Gemini).
# Author: [Your Name/ID]
# =============================================================================

# Install required dependencies quietly
!pip install -q -U google-generativeai pandas openpyxl

import google.generativeai as genai
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import json
import os
import time
from urllib.parse import urlparse

# --- SYSTEM CONFIGURATION ---

# API Configuration
# TODO: Replace the placeholder below with your valid Google Gemini API Key.
GOOGLE_API_KEY = "YOUR_API_KEY_HERE"

# Model Configuration
# Using 'gemini-flash-latest' for stable performance and efficient quota usage.
MODEL_NAME = 'gemini-flash-latest'

# Initialize the Gemini Client
genai.configure(api_key=GOOGLE_API_KEY)


# --- UTILITY FUNCTIONS ---

def get_filename(path_or_url):
    """
    Extracts the clean filename from a local path or web URL.

    Args:
        path_or_url (str): The local file path or HTTP URL.

    Returns:
        str: The extracted filename (e.g., 'image.jpg').
    """
    if path_or_url.startswith('http'):
        return os.path.basename(urlparse(path_or_url).path)
    else:
        return os.path.basename(path_or_url)


def analyze_product_image(image_path_or_url):
    """
    Sends the image to the Multimodal Model to extract visual dimension scores
    and observable attributes.

    Args:
        image_path_or_url (str): Path to local file or URL of the image.

    Returns:
        dict: JSON object containing scores (-5.0 to +5.0) and attributes.
    """
    print(f"⏳ Handling rate limits (2s delay)...")
    time.sleep(2)
    print(f"🔍 Starting analysis for: {image_path_or_url} ...")

    img = None

    # 1. Image Ingestion & Pre-processing
    try:
        # Handle Local Files
        if os.path.exists(image_path_or_url):
            print("   -> Loaded local file successfully.")
            img = Image.open(image_path_or_url)

        # Handle Remote URLs
        elif image_path_or_url.startswith('http'):
            print("   -> Detected web URL. Initiating download...")
            # Use standard browser headers to avoid 403 Forbidden errors
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(image_path_or_url, headers=headers)

            if response.status_code != 200:
                return {"error": f"Download failed with status: {response.status_code}"}

            img = Image.open(BytesIO(response.content))

        else:
            return {"error": f"File not found or invalid path: {image_path_or_url}"}

    except Exception as e:
        return {"error": f"Image processing failed: {str(e)}"}

    # 2. Construct System Instruction (Prompt Engineering)
    # Enforces strict JSON schema and scalar ranges for visual dimensions.
    prompt = """
    Analyze this product image and return a JSON object with visual scores (-5.0 to +5.0).
    Output ONLY raw JSON. No markdown formatting.

    {
        "gender_expression": 0.0,  // -5.0 Masc to +5.0 Fem
        "visual_weight": 0.0,      // -5.0 Light to +5.0 Heavy
        "embellishment": 0.0,      // -5.0 Simple to +5.0 Ornate
        "unconventionality": 0.0,  // -5.0 Classic to +5.0 Avant-garde
        "formality": 0.0,          // -5.0 Casual to +5.0 Formal
        "attributes": ["attr1", "attr2", "attr3"]
    }
    """

    # 3. Model Inference
    try:
        model = genai.GenerativeModel(MODEL_NAME)
        response = model.generate_content([prompt, img])

        # Post-process response to ensure valid JSON parsing
        text = response.text.replace("```json", "").replace("```", "").strip()
        result = json.loads(text)
        return result
    except Exception as e:
        return {"error": f"Inference Error: {str(e)}"}


def save_to_excel(json_result, source_path, filename="product_measurements.xlsx"):
    """
    Persists analysis results to an Excel file, preventing duplicates.

    Args:
        json_result (dict): The output from the analysis function.
        source_path (str): The original path/URL of the image.
        filename (str): The target Excel file name.
    """
    if "error" in json_result:
        print(f"❌ Aborting save: Analysis contained errors.")
        return

    # Data Formatting
    current_filename = get_filename(source_path)
    row_data = json_result.copy()
    row_data['filename'] = current_filename
    row_data['full_source'] = source_path

    # Flatten list attributes for tabular storage
    if "attributes" in row_data and isinstance(row_data["attributes"], list):
        row_data["attributes"] = ", ".join(row_data["attributes"])

    new_row_df = pd.DataFrame([row_data])

    # Persistence Logic (Check Duplicates -> Append)
    try:
        if os.path.exists(filename):
            existing_df = pd.read_excel(filename)

            # Idempotency check: Ensure filename isn't already processed
            if "filename" in existing_df.columns:
                if current_filename in existing_df["filename"].values:
                    print(f"⚠️ duplicate detected: '{current_filename}' already exists in database. Skipping.")
                    return

            # Append new record
            updated_df = pd.concat([existing_df, new_row_df], ignore_index=True)
            updated_df.to_excel(filename, index=False)
            print(f"✅ Record successfully appended to '{filename}'. Total records: {len(updated_df)}")
        else:
            # Initialize new database
            new_row_df.to_excel(filename, index=False)
            print(f"✅ Created new database '{filename}' with initial record.")

    except Exception as e:
        print(f"❌ storage I/O Error: {str(e)}")


# --- MAIN EXECUTION PIPELINE ---

# Define Input Source (Local Path or Web URL)
IMAGE_SOURCE = "/content/image.webp"

# Execute Analysis
analysis_result = analyze_product_image(IMAGE_SOURCE)

# Output Results
print("\n--- ANALYSIS REPORT ---")
print(json.dumps(analysis_result, indent=2))

# Persist Data
save_to_excel(analysis_result, IMAGE_SOURCE)

⏳ Waiting 2 seconds for rate limits...
🔍 Analyzing: /content/image2.webp ...
   -> Detected local file.

--- RESULTS ---
{
  "gender_expression": -1.5,
  "visual_weight": -1.0,
  "embellishment": -2.0,
  "unconventionality": -3.0,
  "formality": 2.0,
  "attributes": [
    "semi-rimless",
    "rectangular frame",
    "metal frame",
    "minimalist design",
    "business appropriate",
    "sleek"
  ]
}
✅ Saved to Excel. Total rows: 2
